## Shopee Code League 2020
<h3>Logistics Challenge</h3>
<b>Team: DragonSnake</b>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta 
import re

In [2]:
%%capture
from tqdm import tqdm
tqdm().pandas()

<h1>1. Explore Dataset</h1>

In [3]:
df = pd.read_csv("data/delivery_orders_march.csv")

In [4]:
df

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...
...,...,...,...,...,...,...
3176308,31504087640510,1585821728,1.585978e+09,NaN,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...
3176309,31504147352227,1585895011,1.586179e+09,1.586232e+09,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10..."
3176310,31504462290482,1585826951,1.585965e+09,NaN,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ..."
3176311,31504851495943,1585891194,1.586230e+09,NaN,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...


The dataset contains over 3 million records of Shopee logistics for each order.
<p>To be specific, the dataset contains 6 columns: [orderid, pick (epoch time in GMT+8), 1st delivery_attempt (epoch time in GMT+8), 2nd delivery_attempt (epoch GMT+8), buyer's address and seller's address].</p>

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3176313 entries, 0 to 3176312
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   orderid              int64  
 1   pick                 int64  
 2   1st_deliver_attempt  float64
 3   2nd_deliver_attempt  float64
 4   buyeraddress         object 
 5   selleraddress        object 
dtypes: float64(2), int64(2), object(2)
memory usage: 145.4+ MB


<b>Column Data Types</b>
<p>Take a closer look in dataframe data types, we can see that the data type of two delivery attempts are float, thus, we need to convert it into date format for later time calculation with SLA</p>

In [6]:
df.isna().sum()

orderid                      0
pick                         0
1st_deliver_attempt          0
2nd_deliver_attempt    1819311
buyeraddress                 0
selleraddress                0
dtype: int64

As indicated above, only the 2nd_deliver_attempt has missing values of over 1.8 million records.

<a name='rules'><h2>Rules Defined by Shopee</h2></a>
<h3>Basic Concepts</h3>
<li>Each orderid represents a distinct transaction on Shopee.</li>
<li>SLA can vary across each route (A route is defined as Seller’s Location to Buyer’s Location) - Refer to SLA_matrix.xlsx</li>
<li>Pick Up Time is defined as the time when the 3PL picks up the parcel and begins to process for delivery. It marks the start of the SLA calculation.</li>
<li>Delivery Attempt is defined as an attempt made by the 3PL to deliver the parcel to the customer. It may or may not be delivered successfully. In the case when it is unsuccessful, a 2nd attempt will be made. A parcel that has no 2nd attempt is deemed to have been successfully delivered on the 1st attempt.</li>
<li>All time formats are stored in epoch time based on Local Time (GMT+8).</li>
<li>Only consider the date when determining if the order is late; ignore the time. <b>Example:</b> df['normalised_date'] = df['dates'].dt.normalize()
</li>
<li>Working Days are defined as Mon - Sat, Excluding Public Holidays.</li>
<li>SLA calculation begins from the next day after pickup (Day 0 = Day of Pickup; Day 1 = Next Day after Pickup)</li>
<li>2nd Attempt must be no later than 3 working days after the 1st Attempt, regardless of origin to destination route (Day 0 = Day of 1st Attempt; Day 1 = Next Day after 1st Attempt).</li>
<br/>
<p>Only consider the date when determining if the order is late; ignore the time.</p>

<p>Assume the following <a name='holidays'>Public Holidays</a>:</p>

<li>2020-03-08 (Sunday);</li>
<li>2020-03-25 (Wednesday);</li>
<li>2020-03-30 (Monday);</li>
<li>2020-03-31 (Tuesday)</li>

<a name='sla'><p><h3>SLA Matrix</h3></p></a>
<img src=https://i.imgur.com/Kxj9sd9.png/>

## 2. Wrangle Data

By default, in order to convert precisely to GMT+8, we need to add 8 hours in addition since pd.to_datetime() function automatically converts to GMT+0

In [7]:
%%time
# Add 8 hours for pick-up time
df['pick'] = df['pick'].apply(lambda x: x + 60*60*8)

# Add 8 hours for 1st attempt
df['1st_deliver_attempt'] = df['1st_deliver_attempt'].apply(lambda x: x + 60*60*8)

# Add 8 hours for 2nd attempt
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].apply(lambda x: x + 60*60*8 if pd.notnull(x) else np.nan)

Wall time: 3.92 s


After successfully converting to GMT+8, let's convert it into calculate-able format (datetime)

In [8]:
%%time
# Convert pick-up time to datetime format
df['pick'] = pd.to_datetime(df['pick'], unit='s')

# Convert 1st deliver attempt to datetime format
df['1st_deliver_attempt'] = pd.to_datetime(df['1st_deliver_attempt'], unit='s')

# Conver 2nd deliver attempt to datetime format
df['2nd_deliver_attempt'] = pd.to_datetime(df['2nd_deliver_attempt'], unit='s')

Wall time: 25.8 s


In [9]:
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02 16:39:57,2020-03-05 13:09:18,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04 16:19:28,2020-03-06 10:53:56,2020-03-10 08:02:56,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04 15:20:34,2020-03-06 09:56:19,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05 22:36:56,2020-03-07 12:45:41,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04 18:38:25,2020-03-06 15:41:40,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


After the conversion, all the time (in epoch) has been converted successfully into GMT+8 timezone. However, as the <a href=#rules> rules </a> defined by Shopee, only the date is taken into consideration for deciding whether it is a late deliver or not. As for that reason, let's just simplify things here by normalize all the times to midnight (00:00 AM).

In [10]:
%%time
# Normalize pick-up date
df['pick'] = df['pick'].dt.normalize()

# Normalize 1st deliver attempt date
df['1st_deliver_attempt'] = df['1st_deliver_attempt'].dt.normalize()

# Normalize 2nd deliver attempt date
df['2nd_deliver_attempt'] = df['2nd_deliver_attempt'].dt.normalize()

Wall time: 231 ms


In [11]:
df.iloc[0]

orderid                                                       2215676524
pick                                                 2020-03-02 00:00:00
1st_deliver_attempt                                  2020-03-05 00:00:00
2nd_deliver_attempt                                                  NaT
buyeraddress           Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...
selleraddress          Pantranco vill. 417 Warehouse# katipunan 532 (...
Name: 0, dtype: object

After conversion, we have managed to removed the details of time from the dataset which leaves only time at the start of the date (00:00 AM).

Let's define <a href=#holidays>public holidays</a> which are mentioned by Shopee as we need to take those into accounts while calculating the data.

In [12]:
# Declare list of public holidays in datetime format
holidays = [pd.Timestamp(2020, 8, 3), pd.Timestamp(2020, 3, 25), pd.Timestamp(2020, 3, 30), pd.Timestamp(2020, 3, 31)]

# Declare day off (sunday):
sunday = 6

<h3>Create a <a href=#sla>SLA Matrix</a></h3>

In [13]:
sla_matrix = np.array([
                      [3, 5, 7, 7], # Metro sla_matrix[0][]
                      [5, 5, 7, 7], # Luzon sla_matrix[1][]
                      [7, 7, 7, 7], # Visayas sla_matrix[2][]
                      [7, 7, 7, 7]  # Mindanao sla_matrix[3][]
                    ])

<h3>Process Addresses</h3>

In [14]:
%%time
# Lowercase buyer's addresses
df['buyeraddress'] = df['buyeraddress'].apply(lambda x: x.lower())

# Lowercase seller's addresses
df['selleraddress'] = df['selleraddress'].apply(lambda x: x.lower())

Wall time: 3.05 s


In [15]:
# Define function to match certain location based on address
def find_location(address, locations=['metro manila', 'luzon', 'visayas', 'mindanao']):
    for loc in locations:
        search_pattern = r"\b({})\b".format(loc)
        if (re.search(search_pattern, address)):
            return loc
    return None

In [16]:
# Define function to map location to matrix index
def map_location(location, locations=['metro manila', 'luzon', 'visayas', 'mindanao']):
    switcher = {value:index for index, value in enumerate(locations)}
    return switcher.get(location)

## Processing And Solving

Index for access and retrieve data in dataframe tuples from itertuples function:
<li>0 - Index</li>
<li>1 - orderid</li>
<li>2 - pick</li>
<li>3 - 1st_deliver_attempt</li>
<li>4 - 2nd_deliver_attempt</li>
<li>5 - buyeraddress</li>
<li>6 - selleraddress</li>

In [17]:
%%time
result_vector = []

# Iterate dataframe
for row in df.itertuples():
    # Find location from buyer address
    buyer_location = find_location(row[5])
    # Map buyer location to matrix index
    buyer_map_index = map_location(buyer_location)
    
    # Find location from seller address
    seller_location = find_location(row[6])
    # Map buyer location to matrix index
    seller_map_index = map_location(seller_location)
    
    # Retrieve shipment days from map indexes (seller to buyer)
    days_to_deliver = sla_matrix[seller_map_index][buyer_map_index]
    
    # Initialize the boolean late label
    is_late = 0
    
    current_time = row[2] # Assign the pick-up date
    day_count = 0 # Assign number of days to a counter
    # Start adding time from the day after the pick-up date till the deliver date
    while (current_time < row[3]): 
        # Check if it is not Sunday
        if(current_time.weekday() != sunday):
            # Check if it is not on holidays
            if(current_time not in holidays):
                current_time += pd.Timedelta(days=1)
                day_count += 1
            else:
                current_time += pd.Timedelta(days=1)
        else:
            current_time += pd.Timedelta(days=1)
    # Check if the delivery date is on time compared to the SLA matrix or not -> If late (1) || or not (0)
    if (day_count > days_to_deliver):
        is_late = 1
    else:
        is_late = 0

    # Check if 2nd deliver attempt is not null and whether the first deliver attempt has been late or not
    if (pd.notnull(row[4]) and is_late == 0):
        current_time = row[3] # Assign the 1st deliver attempt date
        day_count = 0
        while (current_time < row[4]):
            if (current_time.weekday() != sunday):
                if (current_time not in holidays):
                    current_time += pd.Timedelta(days=1)
                    day_count += 1
                else:
                    current_time += pd.Timedelta(days=1)
            else:
                current_time += pd.Timedelta(days=1)
        # If time deliver from first attempt to second attempt is longer than 3 working days -> is late (1) || or not (0)
        if (day_count > 3):
            is_late = 1
        else:
            is_late = 0
    # Append label to vector
    result_vector.append(is_late)


Wall time: 6min 4s


<h2>Output Results</h2>

In [18]:
# Assign result vector to the Dataframe as a Series
df['is_late'] = result_vector

In [19]:
# Retrieve order column and label column from the original Dataframe
result_df = df[['orderid', 'is_late']]
result_df.head()

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0


In [20]:
# Output the result dataframe to a csv file
result_df.to_csv("out.csv", index=False)

<h2>Evaluate Results</h2>

In [21]:
correct_df = pd.read_csv("data/result.csv")

In [22]:
correct_vector = []
for row in correct_df.itertuples():
    correct_vector.append(row[2]) # Add to label column vector

In [23]:
count = 0
for index, value in enumerate(result_vector):
    if(result_vector[index] != correct_vector[index]):
        print("Wrong at index {} which label {} compared to the answer: {}".format(index, result_vector[index], correct_vector[index]))
        count += 1
proportion = (len(result_vector) - count) / len(correct_vector)
print("Number of wrong answers: {}\nKaggle score: {}".format(count, proportion))

Number of wrong answers: 0
Kaggle score: 1.0


<h2>## End of Notebook ##</h2>
<b>Member: Nguyen Pham Quoc Minh</b>
<p><b>Team: DragonSnake</b></p>
<b><a href='https://github.com/bietdoikiem'>Github</a></b>